In [10]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from util.misc import *
from util.batch import *

# Hyper-parameters
task = 7
batch_size=32
seeds = [1]#[1,2,3,4,5,6,7,8,9]
delta=0.05    
binary=True
architecture="resnet"
#epsilons=[0.1]
alphas=[0.3]#[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
image_size=64
#project_folder = "/cephyr/users/adambre/Alvis/mnist_transfer/"
project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
project = 'SNIC2021-7-83'
username = 'adambre'
job = 'batch_train_single.sbatch'

In [12]:
print('Iterating over experiments...\n')
fids = []

os.makedirs(project_folder+'logs', exist_ok=True)



# for s in range(10):
#     seed+=1
#     print(seed)
for seed in seeds:
    np.random.seed(seed)
    for alpha in alphas:
        print("alpha:"+str(alpha))
        fid = project_folder+'logs/batch_t-%d_r-%d_a-%.4f_d-%.4f_b-%d_A-%s_I-%d_B-%d' % \
            (task, seed, alpha, delta, binary ,architecture,image_size, batch_size)

        exp = 'task=%d,seed=%d,alpha=%.4f,delta=%.4f,'% (task, seed, alpha, delta)\
                +'binary=%d,architecture=%s,image_size=%d,batch_size=%d' % (binary, architecture,image_size, batch_size)

        output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

        jobid = int(output[0].split(' ')[-1])
        fid = fid+'-%s' % jobid

        fids.append(fid)

Iterating over experiments...

alpha:0.3


In [23]:
!squeue -u $username

#!squeue -u adambre | awk '{if ($1!=JOBID && $1!=373627) {print $1}}' | tail -n +2 |xargs scancel

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          436844_1     alvis paramete  adambre  R       3:12      1 alvis2-01
          436844_2     alvis paramete  adambre  R       3:12      1 alvis2-05
          436844_3     alvis paramete  adambre  R       3:12      1 alvis2-05
          436844_4     alvis paramete  adambre  R       3:12      1 alvis2-06
          436844_5     alvis paramete  adambre  R       3:12      1 alvis2-06
          436844_6     alvis paramete  adambre  R       3:12      1 alvis2-06
          436844_7     alvis paramete  adambre  R       3:12      1 alvis2-06
          436844_8     alvis paramete  adambre  R       3:12      1 alvis2-06
          436844_9     alvis paramete  adambre  R       3:12      1 alvis2-06
         436844_10     alvis paramete  adambre  R       3:12      1 alvis2-06
         436844_11     alvis paramete  adambre  R       3:12      1 alvis2-06
         436844_12     alvis paramete  adambre  R       3

In [18]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 10 "$fid".out
    print('\n------')
    !tail -n 5 "$fid".err
    print(' \n\n')

------
/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/logs/batch_t-7_r-1_a-0.3000_d-0.0500_b-1_A-resnet_I-64_B-32-436730
tail: cannot open '/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/logs/batch_t-7_r-1_a-0.3000_d-0.0500_b-1_A-resnet_I-64_B-32-436730.out' for reading: No such file or directory

------
tail: cannot open '/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/logs/batch_t-7_r-1_a-0.3000_d-0.0500_b-1_A-resnet_I-64_B-32-436730.err' for reading: No such file or directory
 


